# Réarrengement data Solordinario

In [1]:
import numpy as np
import glob 
import os.path
import scipy.io.wavfile as wavfile

In [2]:
path = 'data'

def listdirectory(path): 
    fichier=[] 
    l = glob.glob(path+'/*') 
    for i in l: 
        if os.path.isdir(i): fichier.extend(listdirectory(i)) 
        else: fichier.append(i) 
    return fichier

files = listdirectory(path)

In [3]:
long = 3 #durée à conserver en sec
for file in files:
    Fe, sig = wavfile.read(file)
    new_dir = 'data_2/'+ file[5:]
    wavfile.write(new_dir, Fe, sig[:long*Fe])

# Dictionnaire avec Pitch et Velocite

In [4]:
import descriptors
import torch
import numpy as np

In [5]:
import warnings
warnings.filterwarnings('ignore')

path = 'data_2'
files = listdirectory(path)
nb_files = len(files)
print(nb_files)
sample_per_sec = 250

dict_env = {'file_name' : [[]]*nb_files, 'env' : [[]]*nb_files, 'vel' : [[]]*nb_files, 'pitch' : [[]]*nb_files, 'corde' : [[]]*nb_files}
f = 0

std_tab = np.zeros(len(files))

for file in files:
    name = file[7:]
    vel = file[14:16]
    corde = file[17:19]
    if file[-6]=='#':
        pitch = file[-7:-4]
    else :
        pitch = file[-6:-4]
        
    Fe, sig = wavfile.read(file)
    sig = sig.astype(np.float32)
    env = np.zeros([2,long*sample_per_sec])
    env[0,:] = descriptors.extract_loudness(sig, sampling_rate=Fe, block_size=int(Fe/250), n_fft=2048)[:long*sample_per_sec]
    env[1,:] = descriptors.extract_pitch(sig, sampling_rate=Fe, block_size=int(Fe/250))[:long*sample_per_sec]
    std_tab[f] = np.std(env[1,:])
    
    dict_env['file_name'][f] = file
    dict_env['env'][f] = env
    dict_env['vel'][f] = vel
    dict_env['pitch'][f] = pitch
    dict_env['corde'][f] = corde
    
    f += 1
  

279


In [7]:
np.save('data_dict.npy', dict_env) 
dict_r = np.load('data_dict_jerem_v1.npy', allow_pickle=True).item()
env_pitch = dict_r['env']

In [8]:
f=0
for env in env_pitch:
    std = np.std(env[1,:])
    if std > 50:
        del dict_r['env'][f]
        del dict_r['file_name'][f]
        del dict_r['vel'][f]
        del dict_r['pitch'][f]
        del dict_r['corde'][f]
        print('delete',f)
    f += 1
        

delete 9
delete 20
delete 23
delete 32
delete 35
delete 55
delete 59
delete 61
delete 80
delete 90
delete 92
delete 111
delete 117
delete 133
delete 137
delete 156
delete 171
delete 180
delete 201
delete 206
delete 236
delete 245
delete 247
delete 248


In [7]:
# Save
np.save('data_dict.npy', dict_r) 
